## Influence and Distribution of different sentences

In this part of experiment, we look inside the distribution of different sentences with their distance of text. 

Because if we only consider the one segment in a sentence, we could not get the right `main sentences` usually, we need to find a more general approach to achieve finding the `main sentences` from a text.

In [1]:
from text_summary import *

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from utlis.get_word_vector_by_glove import get_consistent
from utlis.get_word_vector_by_glove import get_local_vector

In [6]:
import jieba

In [7]:
import numpy as np

In [15]:
def plot_corelation(corelation, sub_plot=None):
    mean = np.mean(corelation)
    _1st_percentile = np.percentile(corelation, 25)
    _3st_percentile = np.percentile(corelation, 75)
    
    if sub_plot:
        plt.subplot(*sub_plot)
    
    print('corelation length is: {}'.format(len(corelation)))
    plt.plot(range(len(corelation)), corelation, c=(0, 0, 0.2))
    plt.fill_between(range(len(corelation)), corelation)
    plt.plot([mean] * len(corelation), 'r--')
    plt.plot([_1st_percentile] * len(corelation), 'g+')
    plt.plot([_3st_percentile] * len(corelation), 'b*')

## 这是一篇广告，不适合做摘要

## this is the distance of first half of ads and tail half of ads

In [ ]:
get_two_sentence_distance("".join(words[: len(words)//2]), "".join(words[len(words)//2:]))

In [10]:
def get_verbose(corelations):
    threshold = 0.03
    variance = np.var(corelations)
    return (1/variance) * np.log(len(corelations))

In [11]:
def normalize(x, old_min=0, old_max=10):
    new_x = (x - old_min) / (old_max - old_min) * 100
    return new_x 

In [12]:
def get_coherence(sentences):
    threshold = 0.5
    text = "".join(sentences)
    half_head = text[:len(text)//2]
    half_tail = text[len(text)//2:]
    distance = get_two_sentence_distance(half_head, half_tail)
    print('distance: {}'.format(distance))
    coherence = (1/(distance)) * np.log(len(sentences))
    return coherence

def is_coherent(text):
    coherence = get_coherence(text)
    threshold = 0.03
    return coherence > threshold

In [13]:
def softmax(array):
    array = np.array(array)
    array -= np.max(array, axis=0)
    return np.exp(array) / sum(np.exp(array))

In [14]:
f = 'experiment/test_text.txt'
corelations = get_text_corelation(f)
plot_corelation(softmax(corelations[1]))

NameError: name 'get_text_corelation' is not defined

## Get the distance of Title with each sentences. 

In [16]:
sentences = [s for s, d in corelations[2]]

NameError: name 'corelations' is not defined

In [17]:
title = '早高峰无盖井卡住两辆车 居民盼相关部门解决'

In [18]:
def get_title_distance(title, sentences):
    return get_text_sentences_distances(title, sentences)

In [19]:
dis = get_title_distance(title, sentences)

NameError: name 'sentences' is not defined

In [20]:
title_corelations = softmax([1 - d for _ , d in dis])

NameError: name 'dis' is not defined

In [21]:
plot_corelation(title_corelations)

NameError: name 'title_corelations' is not defined

In [22]:
plot_corelation(softmax(corelations[1]))

NameError: name 'corelations' is not defined

## Conclusion

The corelation with all the text and title is very similar, at least for "experiment/test_text.txt" this file.

## The furthure test. 

In order to get the more general information, we need to test more texts.

### 1. Test of bounch texts.

In [23]:
files = ['experiment/ads.txt', 'experiment/test_text.txt', 'experiment/test_text_01.txt', 'experiment/test_text_02.txt', 'experiment/test_text_03.txt', 'experiment/test_text_04.txt', 'experiment/text_06.txt', 'experiment/text_07.txt', \
         'experiment/many_verbose.txt', 'experiment/not_coherent.txt', 'experiment/test_wechat.txt']
titles = ['再不去闯，梦想永远只是梦想', '早高峰无盖井卡住两辆车 居民盼相关部门解决', '使馆区武警目击作案过程 围堵拦截小偷', '七国集团峰会联合公报涉及东海、南海问题 中国外交部回应 快看', '端午小长假高速不免费 9个收费站可用支付宝缴费', '轻松一刻：全班15对情侣，这才是最虐狗的班级', \
         '山东省检察院派员出庭于欢故意伤害案二审法庭（出庭意见书全文）', '除了四川人民 最爱打麻将的竟是土耳其', '处女座剧组人设崩塌？穿帮+广告毁了《欢乐颂2', '比亚迪强大中国车王朝擂台赛杭州站开赛', \
         '这个国家每年“六一”前都会给老百姓发钱去度假']
assert len(files) == len(titles)

In [24]:
def plot_one(f, t, index = 0, f_number=1, fig=None):
    fig = fig or plt.figure(figsize=(20, 8))
    print("{}, {}".format(f, t))
    
    corelations = get_text_corelation(f)
    sentences = [s for s, d in corelations[2]]
    coherence = get_coherence(sentences)
    print('coherence == {}'.format(coherence))

    fig.add_subplot(f_number, 3, index * 3 + 1)
    plot_corelation(softmax(corelations[1]))
    title_distance = get_title_distance(t, sentences)
    title_corelation = [1 - d for _, d in title_distance]
    fig.add_subplot(f_number, 3, index * 3 + 2)
    plot_corelation(softmax(title_corelation))
    
    fig.add_subplot(f_number, 3, index * 3 + 3)
    plot_corelation(softmax(corelations[1]))
    plot_corelation(softmax(title_corelation))
    plt.show()

In [25]:
def plot_mutiply_f_t(f_t):
    fig = plt.figure(figsize=(20, 15))
    index = 0
    for f, t in f_t:
        plot_one(f, t, index, f_number=len(f_t), fig=fig)
        index += 1

In [26]:
f_t = list(zip(files, titles))

In [27]:
plot_mutiply_f_t(f_t)

experiment/ads.txt, 再不去闯，梦想永远只是梦想


NameError: name 'get_text_corelation' is not defined

In [ ]:
plot_one(*f_t[0])

## Get Accumulate Corelation Function

In [28]:
def accumulate(x):
    x = np.array(x)
    acc = [np.sum(x[:(index+1)]) for index in range(len(x))]
    return acc

In [29]:
accumulate([1, 2, 3]) == [1, 3, 6]

True

In [30]:
def get_complex_corelation(title_corelation, content_corelation):
    p = 0.5
    return softmax(p * title_corelation + (1 - p) * content_corelation)

In [31]:
file_index = files.index('experiment/text_07.txt')
complex_corelation = get_one_file_complex_corelation(files[file_index], titles[file_index])
sentences = get_text_sentence(files[file_index])

print('most trivial ')
for index, s in enumerate(sentences):
    if complex_corelation[index] < np.percentile(complex_corelation, 25):
        print(s)
        
print('most important')
for index, s in enumerate(sentences):
    if complex_corelation[index] > np.percentile(complex_corelation, 75):
        print(s)

NameError: name 'get_one_file_complex_corelation' is not defined

In [33]:
import logging

In [32]:
def is_outliner(x, array):
    _1st_percentile = np.percentile(array, 25)
    _3th_percentile = np.percentile(array, 75)
    threshold = 1.5
    if (_1st_percentile / x) > threshold or (x / _3th_percentile) > threshold:
        return True
    else:
        return False

In [34]:
def clean_outliner(Xs):
    Xs = np.array(Xs)
    Xs = list(filter(lambda x: not is_outliner(x, Xs), Xs))
    return Xs

In [35]:
clean_outliner([1, 2, 1, 43, 12, 4, 14, -1, 999])

[1, 2, 1, 12, 4, 14, -1]

In [36]:
def l_2_loss(y_hats, ys):
    def f(x):
        max_length = 1000
        if x > max_length: return max_length
        else: return x
    return np.sum(np.square(y_hats - ys))  * 1 / f(len(y_hats))

def have_main_point(complex_corelation, plot=True):
    if len(complex_corelation) < 10: 
        return False, -1
    else:
        print('before clean length is: {}'.format(len(complex_corelation)))
        complex_corelation = clean_outliner(complex_corelation)
        print('end clean legnth is: {}'.format(len(complex_corelation)))
        k = (1.0 - 0) / (len(complex_corelation) - 0)
        ys = k * np.arange(0, len(complex_corelation))
        acc = accumulate(complex_corelation)
        if plot: plot_corelation(acc)    
        l2_loss = l_2_loss(acc, ys)
        threshold = 5.0e-4
        logging.info(l2_loss)
        print("l2-loss: {}".format(l2_loss))
        if l2_loss < threshold:
            return False, l2_loss
        else:
            return True, l2_loss

In [ ]:
def fit_to_summary

In [37]:
def test_if_one_file_fit_summary(text, title):
    mini_length = 200
    if not os.path.isfile(text) and len(text) < mini_length: 
        return False, -1
    else:
        complex_corelation = get_one_file_complex_corelation(text, title)
        return fit_to_summary(complex_corelation)

In [38]:
def distinct(array):
    reversed_array = array[::-1]
    for index, e in enumerate(reversed_array[:-1]):
        if e in reversed_array[index+1:]: array[-(1+index)] = None
    array  = [e for e in array if e is not None]
    return array

def get_main_sentene_with_theme_corelation(sentences, corelations):
    print(sorted(zip(sentences, corelations), key=lambda x: x[1], reverse=True))
    important = clean_outliner(corelations)
    avg_length = np.mean([len(sentence) for sentence in sentences])
    max_sentences_num = int(200/avg_length)
    important = distinct(important)
    most_important = np.sort(important)[::-1][:max_sentences_num]
    most_important_sentences = distinct([s for i, s in enumerate(sentences) if corelations[i] in most_important])
    print(most_important)
    return most_important_sentences

In [172]:
def in_same_sentence(subsentence1, subsentence2, text):
    begin_index = text.index(subsentence1)
    if find_complete_sentence(subsentence1, text[begin_index:]) == find_complete_sentence(subsentence2, text[begin_index:]):
        return True
    else:
        return False

    
def get_complete_sentences_with_corelations(sentences, single_subsentence_corelations, text):
    complete_sentences = []
    complete_sentences_corelations = []

    corelations = [None if is_outliner(x, single_subsentence_corelations) else x for x in single_subsentence_corelations]
    single_complete_corelations = [corelations[0]]
    single_complete_sentence = [sentences[0]]
    
    for index in range(1, len(sentences)):
        sub = sentences[index]
        last_word = single_complete_sentence[-1]
        if in_same_sentence(last_word, sub, text):
            single_complete_sentence.append(sub)
            single_complete_corelations.append(corelations[index])
        else:
            complete_sentences.append(single_complete_sentence)
            complete_sentences_corelations.append(single_complete_corelations)
            
            single_complete_sentence = [sub]
            single_complete_corelations = [corelations[index]]
            
        if index == len(sentences) - 1:
            complete_sentences.append(single_complete_sentence)
            complete_sentences_corelations.append(single_complete_corelations)
    
    return zip(complete_sentences, complete_sentences_corelations)

In [40]:
def get_summary_with_fit_summary_test(text, title):
    ## get the main sentences of one text
    ## if no main sentene or not fit to get summary, return None else return sentences
    mini_length = 250
    complex_corelation = get_one_file_complex_corelation(text, title)
    if not os.path.isfile(text) and len(text) < mini_length: return None
    elif have_main_point(complex_corelation, plot=False)[0]:
        sentences = get_text_sentence(text)
        main_sentences = get_main_sentene_with_theme_corelation(sentences, complex_corelation)
        print('main sentences: {}'.format(main_sentences))
        text_content = get_text_content(text)
        summary = change_sentences_to_complete_sentences(main_sentences, text_content)
        summary = "".join(summary)
        return summary
    else: return None

In [41]:
def change_sentences_to_complete_sentences(sub_sentences, text):
    entire_sentences = [find_complete_sentence(sub_str, text) for sub_str in sub_sentences]
    entire_sentences = distinct(entire_sentences)
    return entire_sentences

## Anaylsis No-Linear Relationship

In [194]:
def get_text_corelation(text):
    text_sentences = get_text_sentence(text)
    distance_map = get_all_sentences_distance(text_sentences)
    distance_sentence_pair = [(string, distance_map[string]) for string in text_sentences]
    corelation = [1 - d for _, d in distance_sentence_pair]
    segments_with_index = [index_word for index_word in enumerate(text_sentences)]
    return segments_with_index, corelation, distance_sentence_pair


def get_one_file_complex_corelation(text, title):
#    print("{} {}".format(text[:50], title))
        
    corelations = get_text_corelation(text)
    sentences = [s for s, d in corelations[2]]
    title_distance = get_title_distance(title, sentences)
    title_corelation = softmax([1 - d for _, d in title_distance])
    content_corelation = softmax(corelations[1])
    complex_corelation = (title_corelation, content_corelation)
    #plot_corelation(complex_corelation[0])
    return complex_corelation[0]

def get_summary_with_nolinear(text, title):
    complex_corelation = get_one_file_complex_corelation(text, title)
    sentences = get_text_sentence(text)
    complete_no_linear = get_complete_sentences_with_corelations(
        sentences, complex_corelation, get_text_content(text, escape_english=False))

    def f(array, total_words_length):
        array = list(filter(lambda x: x is not None, array))
        result = np.mean(array) * (1.05 ** (total_words_length-1))
        return result if not np.isnan(result) else -1

    def get_merged_corelation(single_nolinear_corelations, sentences):
        total_words_length = len("".join(sentences))
        merged_corelation = f(single_nolinear_corelations, total_words_length)
        return merged_corelation

    def get_sentence_and_merged_corelation(subsentences, corelation):
        return (" ".join(subsentences), corelation)

    completed_sentences_with_corelations = []
    
    for s, c in complete_no_linear:
        merged_corelation = get_merged_corelation(c, s)
        single_completed_sentence_with_corelation = get_sentence_and_merged_corelation(s, merged_corelation)
        completed_sentences_with_corelations.append(single_completed_sentence_with_corelation)

    _25_percentile = np.percentile([c for s, c in completed_sentences_with_corelations], 25)
    _60_percentile = np.percentile([c for s, c in completed_sentences_with_corelations], 60)

    sentences = list(map(lambda x: x[0], completed_sentences_with_corelations))
    corelations = list(map(lambda x: x[1], completed_sentences_with_corelations))
    corelations = [x if not np.isnan(x) else -1 for x in corelations]

    total_sentence = []
    total_length = 0
    min_single_length = 3
    for string, corelations in zip(sentences, corelations):
        if corelations > _60_percentile:
            if len(string) >= min_single_length:
                total_length += len(string)
                total_sentence.append(string)
    
    return "。".join(total_sentence)

def get_suitable_length_summary(text, title, fit_length):
    summary = get_summary_with_nolinear(text, title)
    if len(summary) > fit_length:
        return get_summary_with_nolinear(summary, title)
    else:
        return summary
    
def readable_summary(text, title):
    fit = test_if_one_file_fit_summary(text, title)
    if fit:
        fit_length = get_fit_length(len(get_text_content(text)))
        return title + ": " + get_suitable_length_summary(text, title, fit_length)
    else:
        return None

In [70]:
def get_fit_length(original_length):
    length_map = {
        lambda x: x > 1000: 250,
        lambda x: 500 < x < 1000: 200,
        lambda x: 300 < x < 500: 150, 
        lambda x: x < 300: 100
    }
    for cond in length_map:
        if cond(original_length):
            return length_map[cond]

In [180]:
target_file_path = 'experiment/error_analysis.txt'
title = '“天面”整治让平遥古城更古色古香'

In [148]:
content = get_text_content(target_file_path, escape_english=False)

In [151]:
content.index('22日')

29

In [156]:
content[32]

'晚'

In [149]:
find_complete_sentence('22日', content)

'中新网5月24日电 据外媒24日报道，警方称，3名男子疑与22日晚的曼彻斯特体育场爆炸袭击有关联，在曼彻斯特南部地区被捕。'

In [82]:
original_length = len(get_text_content(target_file_path))

In [83]:
get_fit_length(original_length)

150

In [189]:
summary = readable_summary(target_file_path, title)
print(summary)

experiment/error_analysis.txt


/Users/kouminquan/anaconda/envs/env-3/lib/python3.4/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in double_scalars
/Users/kouminquan/anaconda/envs/env-3/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kouminquan/anaconda/envs/env-3/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


花旗：部分中国国企几乎肯定会被穆迪调降评级: 整治与古城风貌不协调的彩钢瓦天面、红色墙面、防水屋面 工程喷涂电力箱体设施 清理屋顶乱堆乱放杂物 连日来 平遥县疏堵结合、积极推进“天面”整治工程 让古城更显古色古香。在此基础上 组织规划、城管、文物等部门50余人 对古城内不协调建筑天面的位置、业主、面积、材料等情况 进行了全面的大排查、大摸底 并根据蓝色彩钢瓦天面、红色墙面、SBS防水屋面属性 逐户建立台账 销号管理。在工程实施过程中 平遥县将古城分成东南西北4大片区 组织4支专业施工队伍 按照先城墙目击范围、后中心区域 先主干街道、后小街小巷的顺序 由外及里 逐院逐户 分层推进 全方位、立体式推进不协调建筑天面整治工作 根据不同情况、不同材质 进行分类整治。对整院搭建的彩钢瓦、二层及二层以上彩钢瓦屋顶、临街临巷的遮雨遮阳彩钢瓦等严重影响古城风貌的 坚决予以了拆除 对院内搭设的遮雨遮阳彩钢瓦、建筑屋顶铺设的防水屋面及红色墙面及电力箱体设施 进行了工程喷涂或更换砖灰色彩钢顶


In [ ]:
import csv
csv_file = 'data_preprocess/updated_news/sqlResult_1262716_0524.csv'
r = csv.reader(open(csv_file))
lines = 

In [ ]:
import csv
csv_file = 'experiment/test_if_fit_summary_with_artifical.csv'
r = csv.reader(open(csv_file))
lines = [l for l in r]
for line in lines[1:]:
    title = line[1]
    content = line[2]
    fit = test_if_one_file_fit_summary(content, title)
    line[4] = fit[1]
    
writer = csv.writer(open(csv_file, 'w', newline=''))
writer.writerows(lines)

In [ ]:
fit_to_summary(complex_corelation)[1]

In [ ]:
import pandas as pd

In [ ]:
file_path = 'experiment/fit_summary_test.txt'

In [ ]:
def get_num_from_my_file(file, mark):
    lines = [line for line in open(file).readlines() if line.startswith(mark)]
    ids = [int(line.strip()[len(mark):]) for line in lines]
    return ids

In [ ]:
IDs = get_num_from_my_file(file_path, 'ID: ')

In [ ]:
fits = get_num_from_my_file(file_path, 'fit:')

In [ ]:
len(fits)

In [ ]:
len(ids) == 50

In [195]:
import csv
import random
test_file = 'test_summary_0613.csv'
csv_file = 'data_preprocess/updated_news/sqlResult_1262716_0524.csv'
contents = pd.read_csv(csv_file)
contents = contents.iterrows()
differents = []
test = []
length = 2
with open(test_file, 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(['id', 'title', 'content', 'description', 'score'])
    for C in contents:
        if length <= 0: break
        ID, content, title = C[1][0], C[1][4], C[1][2]
        if random.random() < 0.8: continue
        summary = readable_summary(content, title)

        if summary:
            spamwriter.writerow([ID, title, content, summary, ''])
            length -= 1
            print(length)

NameError: name 'fit_to_summary' is not defined

In [ ]:
title = '黄小蕾控诉迪士尼工作人员故意刁难 工作态度冷漠'

In [ ]:
test_if_one_file_fit_summary(content, title)

In [ ]:
import numpy as np

In [ ]:
import csv

In [ ]:
differents

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
len(test)

In [ ]:
from collections import Counter
counter = Counter(fits)

In [ ]:
counter[0] / counter[1]

In [ ]:
accuracy_score(fits, test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(fits, test))

## threshold == 1e-3

## threshold == 1.5e-3

In [ ]:
false_nagative = 7
false_positive = 5